# Manipulating font data

By [Allison Parrish](https://www.decontextualize.com/)

This is a short notebook showing some techniques for loading data from TrueType/OpenType font files with [Flat](https://xxyxyz.org/flat) and then doing weird things with it using [Bezmerizing](https://github.com/aparrish/bezmerizing/). Make sure you've gone through these introductory notebooks before proceeding:

* [Flat and asemic writing](flat-asemic-writing.ipynb)
* [Lines and asemic writing](lines-and-asemic.ipynb)

We'll start with some introductory information about typography and font files.

## Typography terminology

First off, there are a number of terms that describe certain distances between parts of letterforms:

![Diagram of typography line terms](http://comptypo.decontextualize.com/public/images/800px-Typography_Line_Terms.svg.png)

Units of measurement:

* Point (1/72nd of an inch)
* Em (the width of a capital 'M' in a font)
* Pixel (the distance from one side of a pixel in a display to the other; may
  differ horizontally and vertically; some LCDs can have meaningful 'sub-pixel'
  addressing; "pixel" can mean something different on high-density displays)

Letter forms by their nature are analog (formed from continuous curves). *Rasterization* is the process of "sampling" the analog curves to a grid, for (e.g.) display on a screen.

## Components of TrueType/OpenType fonts

The most common digital formats for fonts are TrueType (`.ttf`) and OpenType (`.otf`), two closely related standards defined and implemented by [a dizzying amalgamation of Adobe, Apple and Microsoft during the 1990s and early
2000s](https://en.wikipedia.org/wiki/OpenType#History). Both are variants of the [SFNT format](https://en.wikipedia.org/wiki/SFNT) and share many of the same abstractions but differ in many particulars. I attempt to explain the important components of both formats in this section.

### Characters and glyphs

A text is often said to be made up of a string of characters. A character is the basic unit of writing: they're what you learn when you learn to write.

The most important part of a font file is the "glyphs," which are the individual, distinct marks that the font can produce. In many cases, there is a one-to-one correspondence between glyphs and characters (e.g., the glyph *c*
represents the character `c`) but some glyphs represent multiple characters (e.g., ligatures like ﬁ) and some characters may be represeted by more than one glyph (stylistic or contextual variants, e.g., the character `s` in early modern English, represented as *ſ* when found in the middle of a word, and *s* when found at the end).

For this reason, TTF/OTF fonts include information on how to map characters to glyphs. The most straightforward way to do this is with the [cmap table](https://docs.microsoft.com/en-us/typography/opentype/spec/cmap), which links a character (given as, e.g., Unicode code points) to a numbered glyph in
the font. [Substitution features](http://ilovetypography.com/OpenType/opentype-features.html) in OpenType allow the font designer to specify that certain sequences of characters should be represented with particular glyphs (like ligatures, but this feature also be creatively misused; see [Sans Bullshit Sans](https://pixelambacht.nl/2015/sans-bullshit-sans/) and [Doggy](https://www.swiss-miss.com/2012/09/doggy.html)). A font file may also specify other visual aspects of how the text should be rendered, like kerning (adjusting the horizontal and/or vertical space between pairs or longer sequences of characters), which in OpenType is specified in the [GPOS table](https://docs.microsoft.com/en-us/typography/opentype/spec/gpos).

Another aspect of appearance often specified in a font is [hinting](https://en.wikipedia.org/wiki/Font_hinting), which specifies how the vector data in the font should be adjusted during the rasterization process (i.e., displaying on a pixel-based screen), to ensure that the important parts of each glyph are displayed even when interpolated. The TrueType font format has [an absurdly sophisticated system for font hinting](https://docs.microsoft.com/en-us/typography/truetype/hinting), including a Turing-complete programming language, which has led to some [truly
spectacular security vulnerabilities](https://www.youtube.com/watch?v=jioFhWanFxU).

### Shape information

In TrueType and OpenType fonts, the shape of a glyph is specified using data in an sfnt table (called either `glyf` or `CFF `). The point of the data is to define the lines and curves that form the outline of the shapes. The actual formats of these tables are quite frankly baroque and overly complicated; see the [`glyf` specification](https://developer.apple.com/fonts/TrueType-Reference-Manual/RM06/Chap6glyf.html) and the [`CFF `
specification](https://docs.microsoft.com/en-us/typography/opentype/spec/cff). For our purposes, it's not worth talking about the details, since the library we'll be using to parse the font files will smooth everything over for us.

In general, the outline of a glyph is comprised of a series of one or more contours (shapes), which are in turn comprised of lines and curves (with 2D coordinates). In TrueType and OpenType fonts, the start/end positions are
integers on a grid whose resolution is defined by the font itself (e.g., 2048 units per em), with the origin generally at the lower left-hand corner of the glyph.

This illustration from the [FreeType documentation](https://www.freetype.org/freetype2/docs/glyphs/index.html) names
some of the metrics associated with a (horizontal) glyph. For example, the character's *bearing* is a measure of how far from the origin the glyph is; the *advance* of a character controls how much the "pen" should be moved forward
after drawing the character; this may be different from the character's *width*.

![OpenType metrics diagram](http://comptypo.decontextualize.com/public/images/metrics.png)

### Curves

A contour specification consists of lines and curves. Representing a line as data is very easy; you just need to indicate the coordinates of the start and end of the segment. Representing a curve is a little bit more difficult, and
there are multiple strategies for accomplishing this. The most common way to represent curves is as [Bézier
curves](https://en.wikipedia.org/wiki/B%C3%A9zier_curve). Named after French engineer [Pierre Bézier](https://en.wikipedia.org/wiki/Pierre_B%C3%A9zier), who used the math behind the curve in industrial design in the 1960s, a Bézier curve consists of two *anchor points* and a series of *control points*.  The anchor points define the beginning and ending of the curve, and the control points define the extent of the curvature. A Bézier curve with one control point is called *quadratic* and one with two control points is called *cubic*. Higher order curves are possible, but because the use of cubic curves is so widespread in software, the term "Bézier" is often used specifically to mean *cubic* Bézier curves. (This reflects the usage in, e.g., p5.js: see [bezier()](https://p5js.org/reference/#/p5/bezier) and [quadraticVertex()](https://p5js.org/reference/#/p5/quadraticVertex) in the
documentation.)

The following demo lets you play with a cubic Bézier's anchor and control points: [(Cubic) Bezier curve demo](https://editor.p5js.org/allison.parrish/sketches/H1QQ3Nt67)

And the demo below does the same for a quadratic curve: [Quadratic curve demo](https://editor.p5js.org/allison.parrish/sketches/HysPl9FT7)

Here's [a helpful introduction to the math behind Bézier curves](https://pomax.github.io/bezierinfo/), how they're represented as data and how to efficiently rasterize them. (You don't need to read or understand all of this in order to make use of Bézier curves, but it's helpful to read at least the first few sections.) The author of that tutorial has also made a [wonderful library for working with Bézier curves in JavaScript](https://pomax.github.io/bezierjs/). 

The distinction between quadratic and cubic curves is important, because TrueType fonts (with `glyf` tables) *only* support quadratic curves, while OpenType fonts (specifically those that use the `CFF ` table) make use of cubic curves. If you want to be able to draw glyphs from either format, you have to be able to draw both kinds of curves.

### Counters and cutouts

Many glyph outlines consist of more than one contour: the letter `j` for example has a contour for the body of the letter and a separate (non-overlapping) contour for the dot.

Additionally, some glyph outlines have have closed [counters](https://en.wikipedia.org/wiki/Counter_(typography)): contours of the glyph that are completely enclosed within another contour. (The "hole" in the letter `O` is an example; a capital `B` has two counters.) Some glyph outlines, like `®`, have contours within contours within contours!

So the question arises: in a glyph outline, how do you know which contours should be filled in, and which should be "cut out"? (Otherwise stated: how do you know which points are "inside" the letter and should be filled in, and
which points are "outside" the letter and should be left blank?)

This is one of the trickier parts of rendering a glyph. TrueType `glyf` fonts use one strategy: if the points in a contour are defined in a clockwise fashion, then the shape should be filled; if they're defined in a counter-clockwise fashion, then the shape should be cut out. In OpenType `CFF ` fonts, contours are considered cutouts if they're contained inside an odd number of other paths. ([Source](https://github.com/opentypejs/opentype.js/issues/347).) In either case, finding out whether a contour should be cut out or filled in depends on doing some math on the points and curves of the contours themselves.

### Variable fonts

Recent versions of the OpenType standard include support for ["variable" fonts](https://simoncozens.github.io/fonts-and-layout/opentype.html#opentype-font-variations). Variable fonts contain multiple "masters" (i.e., different versions of the same glyphs), alongside information about how to interpolate between the points in those masters. The end goal of variable fonts is to make it possible to dynamically adjust (e.g.) the weight of a font (without having to include separate font data for each weight). As of this writing, support for variable fonts remains incomplete (even in some web browsers), and the library we're using to work with TrueType/OpenType files (sadly) does not support variable fonts. But it's worth knowing about! This [Variable Fonts website](https://v-fonts.com/) has a ton of variable fonts you can play around with to get an idea for how they work.

## Code preliminaries

Whew! With all of that in mind, we can start programming. Before we begin, some preliminary imports from Flat and Bezmerizing:

In [1]:
from flat import document, rgb, rgba, font, strike, shape
from flat.command import moveto, quadto, curveto, lineto, closepath
from bezmerizing import Polyline, Path
import numpy as np
from copy import copy

And some functions from `numpy` which we'll use for generating random numbers.

In [2]:
from numpy.random import uniform, normal, choice

Plus our trusty function for displaying Flat drawings inline:

In [3]:
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

## Flat paths

Flat has a concept of a "path," which is a Python list of `command`s. Five such commands are supported:

* `moveto(x, y)`: Moves pen to x, y.
* `lineto(x, y)`: Moves pen to x, y, drawing a line along the way.
* `quadto(x1, y1, x, y)`: Draws a quadratic Bezier curve from current pen position to x, y, using control point x1, y1.
* `curveto(x1, y1, x2, y2, x, y)`: Draws a cubic Bezier curve from current point to x, y, using control points x1, y1 and x2, y2.
* `closepath`: Closes the current subpath.

To draw a path, create a list of `command`s and pass them to a `shape` object's `path` command. For example, to draw a little strange triangle:

In [4]:
d = document(100, 100, 'mm') # create a document 100mm x 100mm (can replace 'mm' with 'pt')
page = d.addpage() # add a page
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(128, 128, 128, 255)).width(4)
my_cmds = [moveto(50, 33),
           lineto(67, 67),
           curveto(60, 50, 40, 50, 33, 67),
           closepath]
tri_path = pen.path(my_cmds)
page.place(tri_path)
show(page) # show the page

A potential drawback to specifying paths like this is that the coordinates are hard-coded, making it difficult to do things like draw the path at a different size or at a different location on the canvas. To mitigate this drawback, we'll want to be able to apply [affine transformations](https://en.wikipedia.org/wiki/Affine_transformation) to paths, such as translation, rotation, scale, etc. Flat natively supports a "transformation matrix" to apply affine transformations, but that support is a little bit clunky and requires you to hand-code even simple transformations. Instead, we'll use the Bezmerizing `Path` object, which has built-in methods for the most common transformations.

> If you need an explanation of affine transformations or the transformation matrix, I recommend this [helpful introduction](https://www.alanzucconi.com/2016/02/10/tranfsormation-matrix/)—not specific to Flat, but the concepts are the same.

To create a `Path` object, call `Path()` with a list of Flat commands:

In [5]:
doodad = Path(my_cmds)

Evaluating the path object shows its contents:

In [6]:
doodad

Path([moveto(x=50.0000, y=33.0000), lineto(x=67.0000, y=67.0000), curveto(x1=60.0000, y1=50.0000, x2=40.0000, y2=50.0000, x=33.0000, y=67.0000), closepath])

To draw a path, pass it to a `shape` object's `.path()` method:

In [7]:
page = document(80, 80, 'mm').addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(128, 128, 128, 255)).width(4)
page.place(pen.path(doodad))
show(page) # show the page

To transform a path, call one of the `Path` object's transformation functions: `.translate()`, `.rotate()`, or `.scale()`. This will return a new `Path` with the transformation applied to copies of the commands in the original path. (The commands in the original path will be unchanged.) The code below shows the effect of the commands, with the transformed shapes drawn with slight transparency:

In [9]:
d = document(80, 80, 'mm')
page = d.addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(128, 128, 128, 255)).width(4)
alpha_pen = shape().stroke(rgba(0, 0, 0, 128)).fill(rgba(0, 0, 0, 64)).width(4)

page.place(pen.path(doodad))

# move the path up and to the left
translated_path = doodad.translate(-20, -20)
page.place(alpha_pen.path(translated_path))

# scale the path by a factor of 1.5x
scaled_path = doodad.scale(1.5)
page.place(alpha_pen.path(scaled_path))

# rotate the path
rotated_path = doodad.rotate(np.pi * 0.125)
page.place(alpha_pen.path(rotated_path))

show(page)

You'll notice that all of these transformations use the origin (i.e., coordinate `(0, 0)`) as the "center" of the transformation, so (e.g.) the "center" of the `rotate()` transformation is in the upper left-hand corner of the screen. For this reason, when specifying paths for use with transformations, programmers often intentionally center the figure around the origin:

In [10]:
doodad_center = Path([moveto(x=0, y=-17),
                      lineto(x=17, y=17),
                      curveto(x1=10, y1=0, x2=-10, y2=0, x=-17, y=17),
                      closepath])

The cell below shows a series of chained transformations on this centered path:

In [12]:
d = document(80, 80, 'mm')
page = d.addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(128, 128, 128, 255)).width(4)
alpha_pen = shape().stroke(rgba(0, 0, 0, 128)).fill(rgba(0, 0, 0, 64)).width(4)

scaled_path = doodad_center.scale(1.5)
rotated_path = scaled_path.rotate(np.pi * 0.125)
translated_path = rotated_path.translate(40, 40)

page.place(alpha_pen.path(translated_path))

show(page)

Example of chaining transformations in a loop:

In [13]:
width = 240
height = 30
n = 20
page = document(width, height, 'mm').addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(128, 128, 128, 255)).width(2)

for i in range(n):
    this_path = (doodad_center
                 .rotate(i*0.1)
                 .scale(0.333)
                 .translate((width/n)*0.5 + (width / n)*i, height/2))
    page.place(pen.path(this_path))

show(page)

## Drawing strings with flat

Flat has a handful of helpful functions for drawing strings to the screen with a given font. You need to download the font file for the font you want to use, put it in the same directory as this notebook, and then call `font.open` as below. We'll use a font from Google's [Noto collection](https://www.google.com/get/noto/).

In [14]:
f = font.open("./NotoSans-Regular.ttf")

The `strike()` call is like `shape()` but for text; `.size()`'s parameters set the font size and leading, respectively; `.frame()` on the placement reflows the text in an invisible rectangle with the given coordinates and size. Once you've created a strike, you can place the text on the page using the `.text()` method, like so:

In [15]:
d = document(80, 80, 'mm')
page = d.addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(128, 128, 128, 255)).width(4)
text_pen = strike(f).color(rgba(0, 0, 0, 255)).size(24, 32)
page.place(text_pen.text("Sphinx of black quartz, judge my vow.")).frame(10, 10, 60, 60)
show(page)

Flat has a fairly sophisticated system for text layout that is [worth looking into](https://xxyxyz.org/flat#tutorial). But in this tutorial, we're more interested at getting at the actual geometry data in the fonts themselves.

## Glyph paths

Fortunately, Flat lets us access the geometry of glyphs in the font directly. Each glyph has an index, and calling the font's `.glyph()` method returns a list of Flat path commands for that glyph, like so:

In [16]:
f.glyph(38)

For the rest of the notebook, I'm going to create Bezmerizing `Path` objects from these lists, which make the data a bit easier to work with:

In [17]:
glyph_path = Path(f.glyph(38))
glyph_path

Path([moveto(x=491.5000, y=-633.0000), quadto(x1=451.0000, y1=-645.0000, x=403.0000, y=-645.0000), quadto(x1=288.0000, y1=-645.0000, x=222.0000, y=-568.0000), quadto(x1=156.0000, y1=-491.0000, x=156.0000, y=-357.0000), quadto(x1=156.0000, y1=-224.0000, x=217.5000, y=-146.5000), quadto(x1=279.0000, y1=-69.0000, x=402.0000, y=-69.0000), quadto(x1=448.0000, y1=-69.0000, x=490.0000, y=-77.0000), quadto(x1=532.0000, y1=-85.0000, x=573.0000, y=-97.0000), lineto(x=573.0000, y=-19.0000), quadto(x1=533.0000, y1=-4.0000, x=490.5000, y=3.0000), quadto(x1=448.0000, y1=10.0000, x=389.0000, y=10.0000), quadto(x1=280.0000, y1=10.0000, x=207.0000, y=-35.0000), quadto(x1=134.0000, y1=-80.0000, x=97.5000, y=-163.0000), quadto(x1=61.0000, y1=-246.0000, x=61.0000, y=-358.0000), quadto(x1=61.0000, y1=-466.0000, x=100.5000, y=-548.5000), quadto(x1=140.0000, y1=-631.0000, x=217.0000, y=-677.5000), quadto(x1=294.0000, y1=-724.0000, x=404.0000, y=-724.0000), quadto(x1=517.0000, y1=-724.0000, x=601.0000, y=-682

Looking at the path data reveals a couple of interesting things. The first is that the path consists mainly of `lineto` and `quadto` commands. A `lineto` command moves the cursor from the current position to the specified position, drawing a line along the way; a `quadto` command draws a quadratic Bézier curve to the specified `x` and `y` point, with a control point at the coordinates specified by `x1` and `y1`. You'll also notice that y-coordinates are mostly negative. That's because glyphs in fonts are generally drawn with the origin of the coordinate system (`0, 0`) in the *lower left-hand* corner.

> A quadratic Bézier curve has one control point; a cubic Bézier has two control points. TrueType fonts (TTFs) use quadratic Bézier curves exclusively; OpenType fonts (generally) use cubic Bézier curves.

The coordinates of points in TrueType and OpenType fonts are always specified as integers. The value of these integers doesn't correspond with any real-world unit; instead, each font must specify a "density" parameter that indicates how many units in the font's coordinate system correspond with an em-width. In Flat, you can access this with the `.density` attribute of the font object:

In [18]:
f.density

1000

This means that there are 1000 "font space" units per em. Broadly speaking, this means that in order to draw this font at 72 point size, you'd need to scale the glyph by 24 / 1000 (0.024):

In [19]:
glyph_path.scale(72 / f.density)

Path([moveto(x=35.3880, y=-45.5760), quadto(x1=32.4720, y1=-46.4400, x=29.0160, y=-46.4400), quadto(x1=20.7360, y1=-46.4400, x=15.9840, y=-40.8960), quadto(x1=11.2320, y1=-35.3520, x=11.2320, y=-25.7040), quadto(x1=11.2320, y1=-16.1280, x=15.6600, y=-10.5480), quadto(x1=20.0880, y1=-4.9680, x=28.9440, y=-4.9680), quadto(x1=32.2560, y1=-4.9680, x=35.2800, y=-5.5440), quadto(x1=38.3040, y1=-6.1200, x=41.2560, y=-6.9840), lineto(x=41.2560, y=-1.3680), quadto(x1=38.3760, y1=-0.2880, x=35.3160, y=0.2160), quadto(x1=32.2560, y1=0.7200, x=28.0080, y=0.7200), quadto(x1=20.1600, y1=0.7200, x=14.9040, y=-2.5200), quadto(x1=9.6480, y1=-5.7600, x=7.0200, y=-11.7360), quadto(x1=4.3920, y1=-17.7120, x=4.3920, y=-25.7760), quadto(x1=4.3920, y1=-33.5520, x=7.2360, y=-39.4920), quadto(x1=10.0800, y1=-45.4320, x=15.6240, y=-48.7800), quadto(x1=21.1680, y1=-52.1280, x=29.0880, y=-52.1280), quadto(x1=37.2240, y1=-52.1280, x=43.2720, y=-49.1040), lineto(x=40.6800, y=-43.6320), quadto(x1=38.3040, y1=-44.712

Note that this *doesn't* mean that drawing the glyph with Flat will result in a glyph that is exactly 72 points tall! You have to take into account the units of the Flat canvas as well. If your Flat canvas is in millimeters, you'd convert these units to millimeters first (1pt = ~0.385mm):

In [20]:
glyph_path.scale(72 / f.density * 0.385)

Path([moveto(x=13.6244, y=-17.5468), quadto(x1=12.5017, y1=-17.8794, x=11.1712, y=-17.8794), quadto(x1=7.9834, y1=-17.8794, x=6.1538, y=-15.7450), quadto(x1=4.3243, y1=-13.6105, x=4.3243, y=-9.8960), quadto(x1=4.3243, y1=-6.2093, x=6.0291, y=-4.0610), quadto(x1=7.7339, y1=-1.9127, x=11.1434, y=-1.9127), quadto(x1=12.4186, y1=-1.9127, x=13.5828, y=-2.1344), quadto(x1=14.7470, y1=-2.3562, x=15.8836, y=-2.6888), lineto(x=15.8836, y=-0.5267), quadto(x1=14.7748, y1=-0.1109, x=13.5967, y=0.0832), quadto(x1=12.4186, y1=0.2772, x=10.7831, y=0.2772), quadto(x1=7.7616, y1=0.2772, x=5.7380, y=-0.9702), quadto(x1=3.7145, y1=-2.2176, x=2.7027, y=-4.5184), quadto(x1=1.6909, y1=-6.8191, x=1.6909, y=-9.9238), quadto(x1=1.6909, y1=-12.9175, x=2.7859, y=-15.2044), quadto(x1=3.8808, y1=-17.4913, x=6.0152, y=-18.7803), quadto(x1=8.1497, y1=-20.0693, x=11.1989, y=-20.0693), quadto(x1=14.3312, y1=-20.0693, x=16.6597, y=-18.9050), lineto(x=15.6618, y=-16.7983), quadto(x1=14.7470, y1=-17.2141, x=13.6244, y=-1

With all of this in mind, we can actually draw the glyph to the screen:

In [21]:
glyph_path = Path(f.glyph(38)).translate(0, f.density).scale(72 / f.density * 0.385)
page = document(40, 40, 'mm').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240)).width(2)
page.place(pen.path(glyph_path))
show(page)

(I translated the figure "down" by the font density to make sure that we get the whole thing on the canvas.) Try replacing `38` with some other number!

Things get a little bit easier if we use points as the measurement unit for the Flat canvas. That way, we don't have to always include the conversion from points to millimeters. A point is 1/72nd of an inch, so a canvas 144pts wide should be about two inches. (Your browser should show this as about two inches on your actual screen!)

In [22]:
glyph_path = Path(f.glyph(38)).translate(0, f.density)
page = document(144, 72, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240)).width(2)
current_x = 0
for size in [9, 12, 14, 24, 36, 48, 72]:
    page.place(pen.path(glyph_path.scale(size/f.density).translate(current_x, 0)))
    current_x += size
show(page)

### The character map and advances

Glyph indices (i.e., the number you pass to the `.glyph()` method) are arbitrary, presumably assigned by the type designer when creating the font, or something? As mentioned above, figuring out which glyphs to use to represent a given string of characters is not a straightforward task. But for the trivial case of looking up the glyph for a single character, there's a shortcut: the `.charmap` attribute of the font object, which maps Unicode codepoints to glyph indices.

For example, to get the glyph for the letter `a`, first find its Unicode codepoint:

In [23]:
ord("a")

97

... and then look it up in the font's `charmap`:

In [24]:
f.charmap[ord("a")]

68

... then pass that value to the `.glyph()` method. Putting it all together in one expression:

In [25]:
f.glyph(f.charmap[ord("a")])

So, to draw a lower-case `a`:

In [26]:
a_glyph = Path(f.glyph(f.charmap[ord("a")]))
a_glyph = a_glyph.translate(0, f.density).scale(72 / f.density)
page = document(72, 72, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240)).width(2)
page.place(pen.path(a_glyph))
show(page)

The font also stores "advances" for each character, i.e., how much the pen should move horizontally after drawing the character. Advances can be positive or negative (e.g., for a glyph that represents a diacritic). In TrueType and OpenType, there are both horizontal and vertical advances, but Flat only makes the horizontal advances available. Access the advances with the font object's `.advances` attribute, which is a dictionary mapping glyph IDs to their advance widths. For example:

In [27]:
f.advances[f.charmap[ord("a")]]

561

These two helper functions get the path commands for a given character and the advance for a given character:

In [28]:
def glyphcommands(f, ch):
    return Path([copy(cmd) for cmd in f.glyph(f.charmap[ord(ch)])])
def advancefor(f, ch):
    return f.advances[f.charmap[ord(ch)]]

The path for the letter `z`:

In [29]:
glyphcommands(f, "z")

Path([moveto(x=431.0000, y=-68.0000), lineto(x=431.0000, y=0.0000), lineto(x=39.0000, y=0.0000), lineto(x=39.0000, y=-58.0000), lineto(x=327.0000, y=-468.0000), lineto(x=56.0000, y=-468.0000), lineto(x=56.0000, y=-536.0000), lineto(x=424.0000, y=-536.0000), lineto(x=424.0000, y=-470.0000), lineto(x=140.0000, y=-68.0000), lineto(x=431.0000, y=-68.0000), closepath])

... and its advance:

In [30]:
advancefor(f, "z")

470

### Application: Glyph complexity analysis

One potential small application of this data is the ability to analyze the font's glyphs. We can get a full list of all of the glyph indices with the `.values()` attribute of `.charmap`:

In [31]:
all_indices = list(f.charmap.values())

(Note that glyph indices are not guaranteed to be contiguous or consecutive.) That gives us the total number of glyphs in the font (or, at least, glyphs that are associated with individual Unicode characters):

In [32]:
len(all_indices)

2794

Let's quickly find out which character in the font is the most complex—i.e., which glyph has the largest number of commands in its path. To do this, we'll make a list that has a tuple with the glyph index and the length of the glyph's path (obtained with the font's `.glyph()` method):

In [33]:
glyph_path_lengths = [(idx, len(f.glyph(idx))) for idx in all_indices]

The resulting data structure looks like this:

In [34]:
glyph_path_lengths[:12]

[(1, 0),
 (2, 0),
 (3, 0),
 (4, 16),
 (5, 12),
 (6, 36),
 (7, 43),
 (8, 42),
 (9, 44),
 (10, 6),
 (11, 12),
 (12, 12)]

... which tells us that the glyph at index 4 has 16 elements in its path; the glyph at index 7 has 43 elements, etc. The following code sorts this list in reverse order by the value in the second element of the tuple, grabbing just the top ten:

In [35]:
glyphs_by_complexity = sorted(glyph_path_lengths, key=lambda x: x[1], reverse=True)[:10]
glyphs_by_complexity

[(2418, 238),
 (571, 168),
 (3018, 152),
 (2653, 146),
 (2439, 128),
 (2791, 126),
 (2438, 119),
 (2443, 116),
 (3171, 112),
 (2458, 109)]

To get the index of just the most complex character:

In [36]:
most_complex_index = glyphs_by_complexity[0][0]

Let's draw it to see what this glyph looks like!

In [37]:
glyph = Path(f.glyph(most_complex_index))
glyph = glyph.translate(0, f.density).scale(72 / f.density)
page = document(72, 72, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240)).width(2)
page.place(pen.path(glyph))
show(page)

Whoa! What is that wonderful thing? In order to find out, we need some way to track the glyph index back to the Unicode character it represents. An easy way to do this is to "flipping" the `.charmap` dictionary, creating a new dictionary with the values as keys and the keys as values. This gives us a rough reverse-lookup, which we'll use to identify the Unicode character associated with the glyph:

In [38]:
unicode_lookup = {v: k for k, v in f.charmap.items()}

(This is a [dictionary comprehension](https://www.python.org/dev/peps/pep-0274/), which works like a list comprehension but has a key/value pair as the predicate expression, and evaluates to a dictionary with the resulting key/value pairs.)

For example, the glyph at index 42 has the following Unicode code point:

In [39]:
unicode_lookup[42]

71

Using Python's `unicodedata` module, we can get the name for this character:

In [40]:
import unicodedata

In [41]:
unicodedata.name(chr(unicode_lookup[42]))

'LATIN CAPITAL LETTER G'

So what's our mystery glyph?

In [42]:
unicodedata.name(chr(unicode_lookup[most_complex_index]))

'CYRILLIC LETTER MULTIOCULAR O'

It's the [multiocular O](https://en.wikipedia.org/wiki/Multiocular_O) (ꙮ), a variant of the Cyrillic letter O found in a handful of medieval manuscripts and also probably the coolest codepoint in Unicode.

### Visualizing glyph complexity

Let's build on this a little bit to make some asemic writing. Or maybe this is more like a text visualization experiment? I'm not sure! I'm going to write some code that "rewrites" a text by replacing each character with a rectangle whose fill color corresponds to the complexity of that character's corresponding glyph, and whose width corresponds to the advance of the character.

We'll use Robert Frost's "The Road Not Taken," [included in this repository as frost.txt](frost.txt).

The code below does a couple of tricky things. Some of those tricky things are layout-related, which we'll cover in more detail in the following section.

In [43]:
width = 360 # 5in
height = 288 # 4in
border = 9 # 1/8in
lineheight = 16 # size of each line
rectsize = 12 # size of each rectangle
page = document(width, height, 'pt').addpage()
yoff = border  
xoff = border
for line in open("frost.txt"):
    line = line.strip() # remove trailing whitespace
    for ch in line:
        path_len = len(glyphcommands(f, ch).commands)
        val = 255 - (path_len * 5)
        advance = advancefor(f, ch) / f.density * lineheight
        brush = shape().nostroke().fill(rgb(val, val, val))
        page.place(brush.rectangle(xoff, yoff, advance, rectsize))
        xoff += int(advance)
    xoff = border
    yoff += lineheight
show(page)

### Rendering a string "by hand"

Using `glyphcommands()` and `advancefor()`, you can draw an entire string on the screen. The most simple technique to render a string of characters in a Roman-derived alphabet (like English):

* Keep track of the current cursor position.
* For each character in the string:
  * Draw the glyph associated with this character at the current cursor position.
  * Move the current cursor position to the right by the amount of the character's advance.

It should be noted that there are many important drawbacks to this approach:

* It only works with left-to-right writing
* Doesn't take into account kerning, ligatures, etc.

Even with simple left-to-right alphabets, there's more to think about than what we've implemented here. For example, this approach doesn't handle things like text alignment, line breaks, margins, indentation, etc. Still, this is a good starting point.

> The task of taking a sequence of characters and turning it into a sequence of glyphs is called "text shaping," and it's a very complicated process, moreso for scripts like Arabic that rely heavily on ligatures. [HarfBuzz](https://en.wikipedia.org/wiki/HarfBuzz) is a well-known open source library for doing text shaping, and it has [Python bindings](https://github.com/harfbuzz/uharfbuzz).

#### One character at a time

The code below implements this simple string layout procedure. It moves the pen by the character's advance on the x axis and draws the next character at the new position, using `translate()` to translate the path commands to the appropriate place.

In [44]:
s = "quartzy foxes"

In [45]:
page = document(300, 90, 'pt').addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 128))
factor = 36 / f.density
cx = 0
for ch in s:
    glyph_path = glyphcommands(f, ch).scale(factor).translate(cx, 60)
    page.place(pen.path(glyph_path))
    cx += advancefor(f, ch) * factor
show(page)

Because we're drawing one character at a time, we can introduce some random variation to the character positions and scale:

In [46]:
page = document(300, 90, 'pt').addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 128))
factor = 36 / f.density
cx = 0
for ch in s:
    glyph_path = (glyphcommands(f, ch)
                  .scale(factor * uniform(0.75, 1.5))
                  .translate(cx + normal(0, 4), 60 + normal(0, 4)))
    page.place(pen.path(glyph_path))
    cx += advancefor(f, ch) * factor
show(page)

We don't have to draw the characters on the screen in a straight line either:

In [47]:
page = document(300, 300, 'pt').addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 128))
factor = 36 / f.density
cx = 0
for ch in s:
    glyph_path = (glyphcommands(f, ch)
                  .scale(factor * uniform(0.75, 1.5))
                  .translate(factor + uniform(300), factor + uniform(300)))
    page.place(pen.path(glyph_path))
    cx += advancefor(f, ch) * factor
show(page)

#### Making one big path

Another approach is to build one larger `Path` object that combines the paths of each of the individual letters, and draw that instead. This has some interesting advantages that we'll look at later on. The following cell defines a function `combine_path()` that takes a font and a string and returns a path that combines the paths for each character in the string:

In [48]:
from itertools import chain
def combine_path(f, s):
    text_paths = []
    cx = 0
    for ch in s:
        glyph_path = glyphcommands(f, ch).translate(cx, 0)
        text_paths.append(glyph_path)
        cx += advancefor(f, ch)
    combined = Path(list(chain(*text_paths)))
    return combined

Call it like this:

In [49]:
qf_path = combine_path(f, "quartzy foxes")

The resulting path has a lot of commands:

In [50]:
len(qf_path.commands)

274

But you can draw it like any other path:

In [51]:
page = document(300, 90, 'pt').addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 128))
factor = 36 / f.density
page.place(pen.path(qf_path.scale(factor).translate(0, 60)))
show(page)

### Modifying paths

Of course, nothing is forcing us to render these glyph paths exactly as they're stored in the font! We can modify them and reinterpret them. Let's take a character like `z`:

In [52]:
z_path_orig = glyphcommands(f, "z")

In [53]:
z_path_orig

Path([moveto(x=431.0000, y=-68.0000), lineto(x=431.0000, y=0.0000), lineto(x=39.0000, y=0.0000), lineto(x=39.0000, y=-58.0000), lineto(x=327.0000, y=-468.0000), lineto(x=56.0000, y=-468.0000), lineto(x=56.0000, y=-536.0000), lineto(x=424.0000, y=-536.0000), lineto(x=424.0000, y=-470.0000), lineto(x=140.0000, y=-68.0000), lineto(x=431.0000, y=-68.0000), closepath])

Here's what it looks like:

In [54]:
page = document(144, 144, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240))
page.place(pen.path(z_path_orig.scale(144/f.density).translate(0, 144)))
show(page)

We can mess around with the path by *modifying the path commands*. For example, the following code loops over each of the commands in the path and adds random noise to the `x` and `y` coordinates:

In [55]:
z_path = glyphcommands(f, "z")
for cmd in z_path.commands:
    if hasattr(cmd, 'x') and hasattr(cmd, 'y'):
        cmd.x += normal(0, 24)
        cmd.y += normal(0, 24)

(The [`hasattr` function](https://docs.python.org/3/library/functions.html#hasattr) checks to see if the object has the given attribute. This is necessary because the `closepath` command does not have an `x` or `y` attribute, so attempting to modify them would raise an `AttributeError`.)

The cell below draws the modified character and the original:

In [56]:
page = document(288, 144, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240))
page.place(pen.path(z_path_orig.scale(144/f.density).translate(0, 144)))
page.place(pen.path(z_path.scale(144/f.density).translate(144, 144)))
show(page)

Using this technique, we can make programmatic changes to any path by modifying the `x` and `y` coordinates of the path command. It's a bit more complicated than this, since a command might be a `lineto`, a `quadto`, or a `curveto`, each of which has different attributes that need to be modified. The following cell defines a function `nudge()` that shifts all of a command's coordinates regardless of the command's type:

In [57]:
def nudge(cmd, x, y):
    c = copy(cmd)
    if type(cmd) in (moveto, lineto, quadto, curveto):
        c.x += x
        c.y += y
    if type(cmd) in (quadto, curveto):
        c.x1 += x
        c.y1 += y
    if type(cmd) == curveto:
        c.x2 += x
        c.y2 += y
    return c

This function returns a *copy* of the command with the modification applied, so we'll use it to build a new `Path` object with the resulting commands.

Now we can modify paths even for characters that have curves, like `b`:

In [58]:
b_path_orig = glyphcommands(f, "b")

In [59]:
b_path_mod = Path([nudge(cmd, x=normal(0, 15), y=normal(0, 15)) for cmd in b_path_orig])

This code constructs a new `Path` object resulting from adding normally-distributed random numbers to the `x` and `y` coordinates of each command (including curve control points). Here's what the result looks like:

In [60]:
page = document(288, 144, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240))
page.place(pen.path(b_path_orig.scale(144/f.density).translate(0, 144)))
page.place(pen.path(b_path_mod.scale(144/f.density).translate(144, 144)))
show(page)

Using a `for` loop makes it easy to stretch characters out, by nudging their `x` coordinate by a multiple of the `x` coordinate:

In [61]:
a_path_orig = glyphcommands(f, "a")
a_path_cmds = []
for cmd in a_path_orig:
    if hasattr(cmd, 'x'):
        a_path_cmds.append(nudge(cmd, x=cmd.x*0.2, y=0))
    else:
        a_path_cmds.append(cmd)
a_path_mod = Path(a_path_cmds)

In [62]:
page = document(288, 144, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240))
page.place(pen.path(a_path_orig.scale(144/f.density).translate(0, 144)))
page.place(pen.path(a_path_mod.scale(144/f.density).translate(144, 144)))
show(page)

Or we can selectively stretch out characters by only modifying points that meet certain criteria. The code in the cell below shifts up all commands whose `y` coordinate lies above 1/3 of the way from the baseline to the ascender height of the font:

In [63]:
a_path_orig = glyphcommands(f, "g")
a_path_cmds = []
mid_y = f.ascender * -0.33
for cmd in a_path_orig:
    if hasattr(cmd, 'y') and cmd.y < mid_y:
        a_path_cmds.append(nudge(cmd, x=0, y=-200))
    else:
        a_path_cmds.append(cmd)
a_path_mod = Path(a_path_cmds)

Here's what it looks like:

In [64]:
page = document(288, 144, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240))
page.place(pen.path(a_path_orig.scale(144/f.density).translate(0, 144)))
page.place(pen.path(a_path_mod.scale(144/f.density).translate(144, 144)))
show(page)

#### Rendering a modified string

The easiest way to render an entire string with modified glyph paths is to put the path modification code into a function, like so:

In [65]:
def stretch_char(f, ch, height=-0.33):
    path_orig = glyphcommands(f, ch)
    path_cmds = []
    mid_y = f.ascender * height
    for cmd in path_orig:
        if hasattr(cmd, 'y') and cmd.y < mid_y:
            path_cmds.append(nudge(cmd, x=0, y=-200))
        else:
            path_cmds.append(cmd)
    path_mod = Path(path_cmds)
    return path_mod

This function has a parameter that lets you set the cutoff for deciding whether a command in the path will be offset or not, based on its distance from the baseline.

To draw the whole string, use the code from the simple string layout function above, calling the function to generate the glyph for each character in the string, advancing by the `advance`:

In [66]:
page = document(400, 90, 'pt').addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 128))
factor = 72 / f.density
cx = 0
for ch in s:
    glyph_path = stretch_char(f, ch, uniform(-1, -0.25)).scale(factor).translate(cx, 90)
    page.place(pen.path(glyph_path))
    cx += advancefor(f, ch) * factor
show(page)

#### Modifying a larger path

For some effects, it's beneficial to apply coordinate transformations across the entire path of the string, instead of character-by-character. For this, use the `combine_path()` function introduced above, which returns one big path with all of the paths for the glyphs corresponding to the characters in the string. In the example below, I modify the `y` coordinate of each command in the path by the sine of the `x` coordinate, giving the resulting path a pleasing wave...

In [67]:
qf_path_orig = combine_path(f, "quartzy foxes")
qf_path_cmds = []
for cmd in qf_path_orig:
    if hasattr(cmd, 'x'):
        qf_path_cmds.append(nudge(cmd,
                                  x=0,
                                  y=np.sin(cmd.x*0.002)*100))
    else:
        qf_path_cmds.append(cmd)
qf_path_mod = Path(qf_path_cmds)

In [68]:
page = document(400, 90, 'pt').addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).fill(rgba(0, 0, 0, 128))
factor = 72 / f.density
page.place(pen.path(qf_path_mod.scale(factor).translate(0, 90)))
show(page)

Yes, we have reinvented [WordArt](https://en.wikipedia.org/wiki/Microsoft_Office_shared_tools#WordArt).

(Note: "Nudging" the control points of Bézier curves doesn't necessarily produce intuitive results, as you've seen in some of the above examples—the resulting curves aren't quite offset in a linear fashion. The "right" way to do this would be to flatten all of the curves to high-resolution polylines, then perform transformations on those coordinates. But the Bezmerizing library doesn't have an easy way to flatten arbitrary paths to polylines yet, so here we are.)

#### Re-interpreting font commands

One final example. Instead of using the original command in a path, we can "reinterpret" that command as something else. The example below draws lines instead of curves, and circles instead of lines, for each command in a path:

In [69]:
from math import sqrt
d = document(288, 144, 'pt')
page = d.addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).nofill()
factor = 144 / f.density
cx = 0
cy = 0
glyph_path = glyphcommands(f, "R").scale(factor).translate(72, 144)
for cmd in glyph_path:
    if type(cmd) == lineto:
        radius = sqrt(pow(cx - cmd.x, 2) + pow(cy - cmd.y, 2)) * 0.5
        page.place(pen.circle((cx + cmd.x) / 2, (cy + cmd.y) / 2, radius))
    elif type(cmd) in (curveto, quadto):
        page.place(pen.line(cx, cy, cmd.x, cmd.y))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y
show(page)

Drawing all commands as circles:

In [70]:
from math import sqrt
d = document(350, 72, 'pt')
page = d.addpage()
pen = shape().nostroke().fill(rgba(0, 0, 0, 64))
factor = 72 / f.density
cx = 0
cy = 0
glyph_path = combine_path(f, "quartzy foxes").scale(factor).translate(0, 72)
for cmd in glyph_path:
    if type(cmd) in (lineto, curveto, quadto):
        radius = sqrt(pow(cx - cmd.x, 2) + pow(cy - cmd.y, 2)) * 0.5
        page.place(pen.circle((cx + cmd.x) / 2, (cy + cmd.y) / 2, radius))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y
show(page)

## Further resources

For a general introduction to digital typography, I highly recommend [Simon Cozen's *Fonts and Layout for Global Scripts*](https://simoncozens.github.io/fonts-and-layout/). The [history chapter](https://simoncozens.github.io/fonts-and-layout//history.html) is especially insightful and well-researched.

More on digital representations of typefaces:

* [UFO](http://unifiedfontobject.org/versions/ufo3/), a contemporary file format for fonts
* [FreeType Glyph
  Conventions](https://www.freetype.org/freetype2/docs/glyphs/index.html) is documentation for FreeType (a widely-used glyph rendering library), but also a good introduction to digital typography in general
* [An Introduction to TrueType Fonts: A look inside the TTF format](https://scripts.sil.org/cms/scripts/page.php?site_id=nrsi&id=IWS-Chapter08)
* [Bézier Curves and Type Design: A Tutorial](https://learn.scannerlicker.net/2014/04/16/bezier-curves-and-type-design-a-tutorial/)
* [Typography in 8 bits: System fonts](https://damieng.com/blog/2011/02/20/typography-in-8-bits-system-fonts)

Artworks engaging font data:

* [Leslie Mezei, Babel Shook](https://www.atariarchives.org/artist/sec7.php)
* [Eduardo Kac, Accident, 1994](http://www.ekac.org/accident.html)
* [Jürg Lehni & Alex Rich, Empty words, 2008](http://juerglehni.com/works/empty-words)
* [Zach Lieberman et al, IQ Font](http://thesystemis.com/projects/iq-font/)
* [BalletFont](https://web.archive.org/web/20160812072806/http://www.creativeapplications.net/openframeworks/balletfont-openframeworks/) ([making-of video](https://youtu.be/-DdAp26lmIU))